# ipynb2pdf: Convert Jupyter Notebooks to A4 PDF

This notebook converts Jupyter notebook (.ipynb) files to A4-sized PDF documents.

## Requirements
- wkhtmltopdf (system package)
- nbformat
- nbconvert

## Usage
1. Install dependencies (run cell below if needed)
2. Set the input file path and options
3. Run the conversion cell

In [ ]:
# Install required packages (uncomment if needed)
# !pip install nbformat nbconvert
# For wkhtmltopdf installation:
# Ubuntu/Debian: !apt-get install -y wkhtmltopdf
# macOS: !brew install wkhtmltopdf

In [ ]:
import os
import nbformat
from nbconvert import HTMLExporter
from subprocess import run, CalledProcessError

In [ ]:
def check_wkhtmltopdf():
    """Check if wkhtmltopdf is installed"""
    try:
        result = run(['wkhtmltopdf', '--version'], capture_output=True)
        return result.returncode == 0
    except FileNotFoundError:
        return False

def convert_ipynb_to_pdf(ipynb_path, output_path=None, margin_mm=25):
    """
    Convert a Jupyter notebook to A4 PDF

    Args:
        ipynb_path: Path to the input .ipynb file
        output_path: Path to the output .pdf file (optional)
        margin_mm: Margin size in millimeters (default: 25)

    Returns:
        Path to the generated PDF file
    """
    # Check if input file exists
    if not os.path.exists(ipynb_path):
        raise FileNotFoundError(f"Input file not found: {ipynb_path}")

    # Check if wkhtmltopdf is installed
    if not check_wkhtmltopdf():
        raise RuntimeError(
            "wkhtmltopdf is not installed. Please install it:\n"
            "  Ubuntu/Debian: !apt-get install -y wkhtmltopdf\n"
            "  macOS: !brew install wkhtmltopdf\n"
            "  Windows: Download from https://wkhtmltopdf.org/downloads.html"
        )

    # Determine output path
    if output_path is None:
        output_path = ipynb_path.replace('.ipynb', '.pdf')

    # Read the notebook
    print(f"Reading notebook: {ipynb_path}")
    with open(ipynb_path, 'r', encoding='utf-8') as f:
        notebook_content = nbformat.read(f, as_version=4)

    # Setup HTML exporter with embed_images enabled
    html_exporter = HTMLExporter()
    html_exporter.exclude_input = False
    html_exporter.embed_images = True

    # Custom CSS for A4 formatting
    custom_css = """
<style>
    /* Page margins and layout */
    body {
        margin: 0;
        padding: 10px;
        font-family: Arial, sans-serif;
    }

    /* Ensure images fit on A4 page */
    img, .output {
        max-width: 90%;
        height: auto;
    }

    /* Compact cell spacing */
    .input, .output {
        margin: 5px 0;
        padding: 5px;
        font-size: 12px;
    }

    /* Code block styling */
    .input_area {
        background-color: #f5f5f5;
        border-left: 3px solid #3b7ea1;
        padding: 10px;
        overflow-x: auto;
    }

    /* Output area styling */
    .output_area {
        padding: 5px;
    }

    /* Prevent page breaks inside code blocks */
    .cell {
        page-break-inside: avoid;
    }
</style>
"""

    # Convert to HTML
    print("Converting to HTML...")
    html_data, _ = html_exporter.from_notebook_node(notebook_content)
    html_data = custom_css + html_data

    # Save HTML temporarily
    html_temp_path = ipynb_path.replace('.ipynb', '_temp.html')
    with open(html_temp_path, 'w', encoding='utf-8') as f:
        f.write(html_data)

    # Convert HTML to PDF using wkhtmltopdf with A4 settings
    print(f"Converting to A4 PDF: {output_path}")
    try:
        run([
            'wkhtmltopdf',
            '--margin-top', f'{margin_mm}mm',
            '--margin-bottom', f'{margin_mm}mm',
            '--margin-left', f'{margin_mm}mm',
            '--margin-right', f'{margin_mm}mm',
            '--page-size', 'A4',
            '--encoding', 'UTF-8',
            '--enable-local-file-access',
            '--disable-external-links',
            '--disable-javascript',
            '--no-stop-slow-scripts',
            html_temp_path,
            output_path
        ], check=True, capture_output=True)
    except CalledProcessError as e:
        print(f"Error during PDF conversion: {e.stderr.decode()}")
        raise
    finally:
        # Clean up temporary HTML file
        if os.path.exists(html_temp_path):
            os.remove(html_temp_path)

    print(f"✓ PDF generated successfully: {output_path}")
    return output_path

## Configuration

Set your input file path and options below:

In [ ]:
# Configuration
input_file = "example.ipynb"  # Change this to your notebook file
output_file = None  # None = auto-generate from input file name
margin_mm = 25  # Page margin in millimeters

## Run Conversion

Execute the cell below to convert your notebook to PDF:

In [ ]:
# Run the conversion
try:
    pdf_path = convert_ipynb_to_pdf(input_file, output_file, margin_mm)
    print(f"\n{'='*60}")
    print(f"SUCCESS! PDF file created at: {pdf_path}")
    print(f"{'='*60}")
except Exception as e:
    print(f"\n{'='*60}")
    print(f"ERROR: {e}")
    print(f"{'='*60}")

## File Upload (for Google Colab or JupyterLab)

If you want to upload a file and convert it, use the cells below:

In [ ]:
# For Google Colab - Upload file
# Uncomment the lines below if using Google Colab
# from google.colab import files
# uploaded = files.upload()
# input_file = list(uploaded.keys())[0]
# print(f"Uploaded file: {input_file}")

In [ ]:
# For Google Colab - Download the generated PDF
# Uncomment the lines below if using Google Colab
# from google.colab import files
# pdf_filename = input_file.replace('.ipynb', '.pdf')
# files.download(pdf_filename)